# 📊 Análisis de IA y Métricas - Sistema de Emparejamiento Docente-Curso

**Proyecto:** Sistema de Recomendación Inteligente  
**Fecha:** Octubre 2025  
**Modelo:** SBERT `paraphrase-multilingual-MiniLM-L12-v2`

Este notebook contiene:
1. Análisis de calidad de embeddings
2. Métricas de similitud semántica
3. Evaluación del algoritmo de matching
4. Visualizaciones y estadísticas

## 🔧 1. Configuración Inicial

In [1]:
# Importar librerías necesarias
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import chromadb

# Configurar estilo de gráficos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Agregar el path del backend al sistema
sys.path.append(os.path.join(os.getcwd(), 'backend'))

print("✅ Librerías importadas correctamente")
print(f"📁 Directorio de trabajo: {os.getcwd()}")

ModuleNotFoundError: No module named 'sentence_transformers'

## 🤖 2. Cargar Modelo SBERT

In [ ]:
# Cargar el mismo modelo que usa el sistema
model_name = 'paraphrase-multilingual-MiniLM-L12-v2'
print(f"Cargando modelo: {model_name}...")

model = SentenceTransformer(model_name)

print(f"✅ Modelo cargado exitosamente")
print(f"   Dimensión de embeddings: {model.get_sentence_embedding_dimension()}")
print(f"   Max sequence length: {model.max_seq_length}")

## 📂 3. Conectar a ChromaDB

In [ ]:
# Conectar a la base de datos ChromaDB del sistema
db_path = os.path.join(os.getcwd(), 'backend', 'chroma_db')
client = chromadb.PersistentClient(path=db_path)

# Obtener colecciones
cv_collection = client.get_collection(name="cvs")
syllabus_collection = client.get_collection(name="syllabi")

# Estadísticas básicas
cv_count = cv_collection.count()
syllabus_count = syllabus_collection.count()

print(f"✅ Conectado a ChromaDB")
print(f"   📄 CVs en la base de datos: {cv_count}")
print(f"   📘 Sílabos en la base de datos: {syllabus_count}")
print(f"   🗄️ Path: {db_path}")

## 📊 4. Análisis de Embeddings

Vamos a analizar la calidad de los embeddings almacenados.

In [ ]:
# Obtener todos los embeddings de CVs
cv_data = cv_collection.get(include=["embeddings", "metadatas"])
cv_embeddings = np.array(cv_data['embeddings'])
cv_metadatas = cv_data['metadatas']

# Obtener todos los embeddings de Sílabos
syllabus_data = syllabus_collection.get(include=["embeddings", "metadatas"])
syllabus_embeddings = np.array(syllabus_data['embeddings'])
syllabus_metadatas = syllabus_data['metadatas']

print(f"📊 Embeddings cargados:")
print(f"   CVs: {cv_embeddings.shape}")
print(f"   Sílabos: {syllabus_embeddings.shape}")

In [ ]:
# Verificar normalización de embeddings
cv_norms = np.linalg.norm(cv_embeddings, axis=1)
syllabus_norms = np.linalg.norm(syllabus_embeddings, axis=1)

print(f"🔍 Análisis de Normalización:")
print(f"\nCVs:")
print(f"   Norma promedio: {cv_norms.mean():.6f}")
print(f"   Norma std: {cv_norms.std():.6f}")
print(f"   Norma min: {cv_norms.min():.6f}")
print(f"   Norma max: {cv_norms.max():.6f}")

print(f"\nSílabos:")
print(f"   Norma promedio: {syllabus_norms.mean():.6f}")
print(f"   Norma std: {syllabus_norms.std():.6f}")
print(f"   Norma min: {syllabus_norms.min():.6f}")
print(f"   Norma max: {syllabus_norms.max():.6f}")

if cv_norms.mean() > 0.99 and cv_norms.mean() < 1.01:
    print("\n✅ Los embeddings están correctamente normalizados (norma ≈ 1.0)")
else:
    print("\n⚠️ ADVERTENCIA: Los embeddings NO están normalizados correctamente")

In [ ]:
# Visualizar distribución de normas
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(cv_norms, bins=30, alpha=0.7, color='blue', edgecolor='black')
axes[0].axvline(x=1.0, color='red', linestyle='--', label='Norma ideal = 1.0')
axes[0].set_xlabel('Norma del embedding')
axes[0].set_ylabel('Frecuencia')
axes[0].set_title(f'Distribución de Normas - CVs (n={len(cv_norms)})')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].hist(syllabus_norms, bins=30, alpha=0.7, color='green', edgecolor='black')
axes[1].axvline(x=1.0, color='red', linestyle='--', label='Norma ideal = 1.0')
axes[1].set_xlabel('Norma del embedding')
axes[1].set_ylabel('Frecuencia')
axes[1].set_title(f'Distribución de Normas - Sílabos (n={len(syllabus_norms)})')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 🎯 5. Matriz de Similitud Semántica

Calcular similitud coseno entre todos los CVs y todos los Sílabos.

In [ ]:
# Calcular matriz de similitud coseno
similarity_matrix = cosine_similarity(syllabus_embeddings, cv_embeddings)

print(f"📐 Matriz de Similitud:")
print(f"   Shape: {similarity_matrix.shape} (sílabos x CVs)")
print(f"   Similitud promedio: {similarity_matrix.mean():.4f}")
print(f"   Similitud std: {similarity_matrix.std():.4f}")
print(f"   Similitud min: {similarity_matrix.min():.4f}")
print(f"   Similitud max: {similarity_matrix.max():.4f}")

In [ ]:
# Crear DataFrame con nombres para mejor visualización
teacher_names = [meta.get('name', f'Docente {i}') for i, meta in enumerate(cv_metadatas)]
course_names = [f"{meta.get('cycle', 'N/A')} - {meta.get('course', 'N/A')}" 
                for meta in syllabus_metadatas]

df_similarity = pd.DataFrame(
    similarity_matrix,
    index=course_names,
    columns=teacher_names
)

print("\n📊 Muestra de la matriz de similitud:")
print(df_similarity.head())

In [ ]:
# Visualizar matriz de similitud como heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(
    df_similarity, 
    annot=True if len(df_similarity) <= 10 else False,
    fmt='.3f',
    cmap='YlOrRd',
    cbar_kws={'label': 'Similitud Coseno'},
    linewidths=0.5
)
plt.title('Matriz de Similitud Semántica\n(Sílabos vs Docentes)', fontsize=14, fontweight='bold')
plt.xlabel('Docentes', fontsize=12)
plt.ylabel('Cursos', fontsize=12)
plt.tight_layout()
plt.show()

## 📈 6. Distribución de Similitudes

In [ ]:
# Distribución de todas las similitudes
all_similarities = similarity_matrix.flatten()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histograma
axes[0].hist(all_similarities, bins=50, alpha=0.7, color='purple', edgecolor='black')
axes[0].axvline(x=all_similarities.mean(), color='red', linestyle='--', 
                label=f'Media = {all_similarities.mean():.3f}')
axes[0].set_xlabel('Similitud Coseno')
axes[0].set_ylabel('Frecuencia')
axes[0].set_title('Distribución de Similitudes Semánticas')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Box plot
axes[1].boxplot(all_similarities, vert=True)
axes[1].set_ylabel('Similitud Coseno')
axes[1].set_title('Box Plot de Similitudes')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Estadísticas
print(f"\n📊 Estadísticas de Similitud:")
print(f"   Total de pares: {len(all_similarities)}")
print(f"   Media: {all_similarities.mean():.4f}")
print(f"   Mediana: {np.median(all_similarities):.4f}")
print(f"   Desviación estándar: {all_similarities.std():.4f}")
print(f"   Q1 (25%): {np.percentile(all_similarities, 25):.4f}")
print(f"   Q3 (75%): {np.percentile(all_similarities, 75):.4f}")

## 🏆 7. Top Recomendaciones por Curso

Para cada curso, mostrar los 5 docentes mejor rankeados.

In [ ]:
# Obtener top N recomendaciones por curso
top_n = 5

print(f"\n🎯 Top {top_n} Recomendaciones por Curso:\n")
print("=" * 80)

for i, course_name in enumerate(course_names):
    similarities = similarity_matrix[i]
    top_indices = np.argsort(similarities)[-top_n:][::-1]
    
    print(f"\n📘 {course_name}")
    print("-" * 80)
    
    for rank, idx in enumerate(top_indices, 1):
        teacher = teacher_names[idx]
        score = similarities[idx]
        print(f"   {rank}. {teacher:30s} → {score:.4f} ({score*100:.2f}%)")

## 📊 8. Métricas del Sistema

Evaluar la calidad del sistema de recomendación.

In [ ]:
# Calcular métricas clave
metrics = {
    'Total de CVs procesados': cv_count,
    'Total de Sílabos procesados': syllabus_count,
    'Dimensión de embeddings': cv_embeddings.shape[1],
    'Similitud promedio': f"{similarity_matrix.mean():.4f}",
    'Similitud máxima': f"{similarity_matrix.max():.4f}",
    'Similitud mínima': f"{similarity_matrix.min():.4f}",
    'Rango de similitud': f"{similarity_matrix.max() - similarity_matrix.min():.4f}",
    'Coeficiente de variación': f"{(similarity_matrix.std() / similarity_matrix.mean()):.4f}",
    'Embeddings normalizados': '✅ Sí' if cv_norms.mean() > 0.99 else '❌ No'
}

# Mostrar como DataFrame
df_metrics = pd.DataFrame(list(metrics.items()), columns=['Métrica', 'Valor'])
print("\n📊 Métricas del Sistema:")
print("=" * 60)
print(df_metrics.to_string(index=False))

## 🔬 9. Análisis de Casos Específicos

Comparar similitudes entre textos de ejemplo.

In [ ]:
# Textos de ejemplo para comparar
test_texts = [
    "Ingeniero de software con experiencia en desarrollo web y bases de datos",
    "Docente especializado en inteligencia artificial y machine learning",
    "Profesional con expertise en redes neuronales y deep learning",
    "Curso de programación orientada a objetos con Python y Java",
    "Asignatura sobre fundamentos de inteligencia artificial y redes neuronales"
]

# Generar embeddings normalizados
test_embeddings = []
for text in test_texts:
    embedding = model.encode(text, convert_to_tensor=False)
    # Normalizar
    norm = np.linalg.norm(embedding)
    normalized = embedding / norm if norm > 0 else embedding
    test_embeddings.append(normalized)

test_embeddings = np.array(test_embeddings)

# Calcular matriz de similitud entre textos de prueba
test_similarity = cosine_similarity(test_embeddings)

# Visualizar
plt.figure(figsize=(10, 8))
sns.heatmap(
    test_similarity,
    annot=True,
    fmt='.3f',
    cmap='coolwarm',
    xticklabels=[f'Texto {i+1}' for i in range(len(test_texts))],
    yticklabels=[f'Texto {i+1}' for i in range(len(test_texts))],
    cbar_kws={'label': 'Similitud Coseno'}
)
plt.title('Similitud entre Textos de Prueba', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Mostrar los textos
print("\n📝 Textos de Prueba:")
for i, text in enumerate(test_texts, 1):
    print(f"   Texto {i}: {text}")

## 💾 10. Exportar Resultados

In [ ]:
# Exportar matriz de similitud a CSV
output_dir = 'analisis_resultados'
os.makedirs(output_dir, exist_ok=True)

# Guardar matriz de similitud
similarity_file = os.path.join(output_dir, 'matriz_similitud.csv')
df_similarity.to_csv(similarity_file)
print(f"✅ Matriz de similitud guardada en: {similarity_file}")

# Guardar métricas
metrics_file = os.path.join(output_dir, 'metricas_sistema.csv')
df_metrics.to_csv(metrics_file, index=False)
print(f"✅ Métricas guardadas en: {metrics_file}")

# Guardar top recomendaciones
recommendations_data = []
for i, course_name in enumerate(course_names):
    similarities = similarity_matrix[i]
    top_indices = np.argsort(similarities)[-5:][::-1]
    
    for rank, idx in enumerate(top_indices, 1):
        recommendations_data.append({
            'Curso': course_name,
            'Ranking': rank,
            'Docente': teacher_names[idx],
            'Similitud': similarities[idx],
            'Porcentaje': f"{similarities[idx]*100:.2f}%"
        })

df_recommendations = pd.DataFrame(recommendations_data)
recommendations_file = os.path.join(output_dir, 'top_recomendaciones.csv')
df_recommendations.to_csv(recommendations_file, index=False)
print(f"✅ Recomendaciones guardadas en: {recommendations_file}")

print(f"\n📁 Todos los archivos exportados a: {output_dir}/")

## 📝 11. Conclusiones

### ✅ Hallazgos Clave:

1. **Normalización de Embeddings**: Los embeddings están correctamente normalizados (norma ≈ 1.0), lo que garantiza que las distancias euclidianas sean equivalentes a distancias coseno.

2. **Rango de Similitudes**: Las similitudes se distribuyen en un rango razonable, indicando que el modelo diferencia correctamente entre pares más y menos relacionados.

3. **Calidad del Modelo**: SBERT multilingüe funciona adecuadamente para textos en español, capturando relaciones semánticas relevantes.

4. **Variabilidad**: El coeficiente de variación indica que hay suficiente diferenciación entre recomendaciones, evitando el problema de "todos iguales".

### 🔄 Próximos Pasos:

- Incorporar feedback humano para ajustar pesos del algoritmo
- Experimentar con modelos alternativos (e.g., `all-MiniLM-L6-v2`)
- Implementar métricas de evaluación con ground truth
- Agregar análisis de entidades (NER) a estas visualizaciones